In [1]:
%display latex

In [11]:
N = 3
F = GF(2^N, 'x')
x = F.gen()

In [12]:
basis = [x^3, x^5, x^6]

In [13]:
curve = [
    (F(0),F(0)), (x^4,F(0)), (x^4,x^5), (x^3,x^7),
    (x^3,x^4), (x^6,x^4), (x^6,x^7), (F(0),x^5)
]

In [14]:
len(curve)

8

In [15]:
for p1 in curve:
    for p2 in curve:
        p3 = (p1[0] + p2[0], p1[1] + p2[1])
        if p3 not in curve:
            raise Exception

In [17]:
def toInt(k):
    return list(F).index(k)

In [18]:
def graph(pts):
    m = zero_matrix(SR, 2^N, 2^N)
    for p in pts:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

In [19]:
graph(curve)

[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0]

In [38]:
def components(k):
    return vector([(k * b).trace() for b in basis])

In [39]:
def assemble(v):
    s = 0
    for i, b in enumerate(basis):
        s += v[i] * b
    return s

In [40]:
for k in F:
    if assemble(components(k)) != k:
        raise Exception

In [47]:
test = Set()

In [49]:
type(test)

<class 'sage.sets.set.Set_object_enumerated_with_category'>

In [51]:
test = set()

In [52]:
type(test)

<class 'set'>

In [53]:
test.add(x)

In [54]:
test

{x}

In [101]:
M_alpha = []
good_alphas = []
for M in Matrix(Integers(2), 3, 3).parent():
    alphas = []
    for tau in F:
        v = components(tau)
        alphas.append(assemble(M * v))
    if set(alphas) == set([p[0] for p in curve]):
        good_alphas.append(alphas)
        M_alpha.append(M)

M_beta = []
good_betas = []
for M in Matrix(Integers(2), 3, 3).parent():
    betas = []
    for tau in F:
        v = components(tau)
        betas.append(assemble(M * v))
    if set(betas) == set([p[1] for p in curve]):
        good_betas.append(betas)
        M_beta.append(M)

print(len(good_alphas), len(good_betas))

42 42


In [102]:
Ms = []
params = []
for i, alphas in enumerate(good_alphas):
    for j, betas in enumerate(good_betas):
        param = list(zip(alphas, betas))
        if set(param) == set(curve):
            params.append(param)
            Ms.append((M_alpha[i], M_beta[j]))
len(params)

168

In [103]:
params[0]

[(0, 0),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (x + 1, x^2 + x),
 (x + 1, 1),
 (x^2 + 1, x^2 + x),
 (0, x^2 + x + 1),
 (x^2 + x, x^2 + x + 1)]

In [104]:
curve

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

In [105]:
b = list()
for p in params:
    if p not in b:
        b.append(p)

In [106]:
len(b)

168

In [107]:
len(Ms)

168

In [111]:
Ms[0][0]

[1 0 0]
[0 0 0]
[0 1 0]

In [112]:
Ms[0][1]

[1 1 0]
[0 0 1]
[1 1 0]

In [113]:
[(tau * x^3).trace() * x^3 + (tau * x^6).trace() * x^6 for tau in F]

[0, x^2 + 1, x + 1, x + 1, x^2 + x, 0, x^2 + 1, x^2 + x]

In [114]:
[p[0] for p in curve]

[0, x^2 + x, x^2 + x, x + 1, x + 1, x^2 + 1, x^2 + 1, 0]

In [123]:
tau = var('tau')

In [126]:
Ms[-1]

(
[0 1 1]  [1 0 0]
[0 0 0]  [1 0 1]
[1 1 1], [1 0 0]
)

Better way...

We can build any of the 168 parametrizations by selecting three generating elements of the curve, $p_1,p_2,p_3 \in \gamma$ and using the following expression:
$$
(\alpha(\tau), \beta(\tau))
= Tr(\tau x^3) p_1 + Tr(\tau x^5) p_2 + Tr(\tau x^6) p_3.
$$

To find the generating elements of the curve we can select two distinct points and then select a third one that is distinct from the first two and is also not equal to their sum.

In [318]:
curve

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

For example we choose the following points:
$$
p_1 = (x^2+x,0), \quad
p_2 = (x+1,1), \quad
p_3 = (x^2+x,x^2+x+1).
$$

In [319]:
def gamma(tau, gens):
    comps = components(tau)
    p = vector([F(0), F(0)])
    for i, g in enumerate(gens):
        p += comps[i] * g
    return tuple(p)

In [320]:
gens = [vector([x^2+x,F(0)]), vector([x+1,F(1)]), vector([x^2+x,x^2+x+1])]

In [321]:
[gamma(tau, gens) for tau in F]

[(0, 0),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (x^2 + x, 0),
 (0, x^2 + x + 1),
 (x + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x + 1, x^2 + x)]

In [322]:
def powers(pts):
    return [(toInt(p[0]), toInt(p[1])) for p in pts]

In [323]:
powers([gamma(tau, gens) for tau in F])

[(0, 0), (6, 4), (6, 7), (4, 0), (0, 5), (3, 7), (4, 5), (3, 4)]

In [253]:
def listminus(rs, notrs):
    res = []
    for s in rs:
        if s not in notrs:
            res.append(s)
    return(res)

generators = []
set1 = listminus(curve,[(F(0),F(0))])
for p1 in set1:
    set2 = listminus(set1,[p1])
    for p2 in set2:
        set3 = listminus(set2, [p2, (p1[0]+p2[0],p1[1]+p2[1])])
        for p3 in set3:
            generators.append([vector(p1), vector(p2), vector(p3)])

In [257]:
len(generators)

168

In [262]:
for c in [[gamma(tau, gen) for tau in F] for gen in generators]:
    if set(c) != set(curve):
        raise Exception

In [266]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

def ray(mu):
    return lambda t: mu * t

def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [267]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

In [268]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

In [269]:
k = 0
for t1 in F:
    for t2 in F:
        p1 = gamma(t1, generators[k])
        p2 = gamma(t2, generators[k])
        d1 = D(*p1)
        d2 = D(*p2)
        d3 = D(*(vector(p1) + vector(p2)))
        if d1 * d2 != d3:
            raise Exception

In [306]:
def eig_proj(k, j):
    eig = zero_matrix(SR, 2^N, 2^N)
    for tau in F:
        eig += chi(tau * k) * D(*gamma(tau, generators[j])) / 2^N
    return eig

In [308]:
eig = eig_proj(F(0), 123)

In [309]:
dop = D(*gamma(x, generators[0]))
dop * eig * dop.conjugate_transpose() == eig

True

In [311]:
for tau in F:
    for k in F:
        eig = eig_proj(k, 123)
        dop = D(*gamma(x, generators[0]))
        if dop * eig * dop.conjugate_transpose() != eig:
            raise Exception

In [312]:
for k in F:
    print(eig_proj(k, 123), '\n')

[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4]
[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4] 

[     0      0      0      0      0      0      0      0]
[     0    1/4  1/4*I -1/4*I      0      0   -1/4      0]
[     0 -1/4*I    1/4   -1/4      0      0  1/4*I      0]
[     0  1/4*I   -1/4    1/4      0      0 -1/4*I      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0   -1/4 -1/4*I  1/4*I      0      0    1/4      0]
[     0      0      0      0      0      0      0      0] 

[     0      0      0      0      0      0      0      0]
[     0   

In [303]:
V = CC^64
V

Vector space of dimension 64 over Complex Field with 53 bits of precision

In [313]:
V.linear_dependence([vector(eig_proj(k, 123).list()) for k in F])

[

]